In [4]:
%matplotlib qt5

In [5]:
import scipy.io
import mne
import h5py
import numpy as np
import hdf5storage

import sys
sys.path.append('/home/lfa-01/Documentos/RepositoriosGit/')

import importlib

import SpikeWaveDetectorClass 
import RippleDetectorClass

In [6]:
from scipy import stats

In [7]:
# FILEPATH: signal
filepath = '/home/lfa-01/Documentos/Data_NgoStaresina_eLife2020_OSF/Raw data/EEGs/pat14_HIPP.mat'
mat_data = hdf5storage.loadmat(filepath)

# Now you can work with the loaded data
times = mat_data['time'][0][0][0]
fsample = mat_data['fsample'][0][0]
ch_label = str(mat_data['label'][0][0][0])
cant_chan = len([ch_label])
ieee_signal = mat_data['trial'][0][0][0]
ieee_signal = np.reshape(ieee_signal, (cant_chan, len(ieee_signal)))

# Convert the loaded data to a MNE-Python object
mne_data = mne.io.RawArray(ieee_signal, info=mne.create_info(ch_names=[ch_label], sfreq=fsample, ch_types=['seeg']*cant_chan))


Creating RawArray with float64 data, n_channels=1, n_times=28799616
    Range : 0 ... 28799615 =      0.000 ... 28799.615 secs
Ready.


In [9]:
# FILEPATH: events info
filepath_evt = '/home/lfa-01/Documentos/Data_NgoStaresina_eLife2020_OSF/Raw data/Ripples/pat14_HIPP_ripples.mat'
mat_evt_info = hdf5storage.loadmat(filepath_evt)

In [10]:
keys = mat_evt_info.keys()
print(keys)

print(type(mat_evt_info['EvtInfo']))
print(mat_evt_info['EvtInfo'].shape)
print(mat_evt_info['EvtInfo'][0][3][0])
print(len(mat_evt_info['EvtInfo'][0][3][0]))

dict_keys(['EvtInfo', 'EvtSummary', 'EvtThres', 'artifactPadding', 'bpfreq', 'center', 'criterionCenter', 'criterionLen', 'criterionPadding', 'criterionScale', 'criterionVar', 'detectCh', 'doubleThres', 'envelopeType', 'envelopeWin', 'filtType', 'findEvtFree', 'mergeEvts', 'minCycles', 'minCyclesNum', 'rejects', 'searchIndivEvtPeak', 'slStages', 'smoothEnv', 'smoothEnvWin', 'thresCh', 'thresType', 'upperCutoff', 'upperCutoffThres', 'variance', 'zscore'])
<class 'numpy.ndarray'>
(1,)
[3.8320000e+03 7.0520000e+03 2.8194000e+04 3.2839000e+04 3.3894000e+04
 5.8330000e+04 7.4028000e+04 9.2741000e+04 1.0388700e+05 1.0404900e+05
 1.0712700e+05 1.1698700e+05 1.2361200e+05 1.5897800e+05 1.6360300e+05
 1.6534100e+05 1.6857800e+05 1.7896700e+05 1.8709200e+05 2.1130700e+05
 2.1454400e+05 2.2140900e+05 2.2291300e+05 2.2817100e+05 2.2879000e+05
 2.2962000e+05 2.3947300e+05 2.4529500e+05 2.5454700e+05 2.6366700e+05
 3.2586900e+05 3.2888900e+05 3.2909400e+05 3.5453200e+05 3.5705500e+05
 3.6609700e+05 

In [11]:
print(mne_data.info)

<Info | 7 non-empty values
 bads: []
 ch_names: ['HIPP']
 chs: 1 sEEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 1
 projs: []
 sfreq: 1000.0 Hz
>


In [12]:
orig_ripples_detect = mat_evt_info['EvtInfo'][0][3][0] # del struct EvtInfo, el tercer vector es el punto medio de los ripples identificados
events_array_orig = np.column_stack( (orig_ripples_detect, np.zeros(len(orig_ripples_detect)), np.ones(len(orig_ripples_detect))))

events_array_orig = events_array_orig.astype(int)
print(events_array_orig)

events_id_orig = {'putative_ripple_Ngo': 1}
event_color_orig = {'putative_ripple_Ngo': 'm'}

[[    3832        0        1]
 [    7052        0        1]
 [   28194        0        1]
 ...
 [27105165        0        1]
 [27135802        0        1]
 [27135888        0        1]]


In [13]:
seeg_channels = mne.pick_types(mne_data.info, seeg=True)
mne_data.plot(duration=30, order=seeg_channels, n_channels=len(seeg_channels),
         remove_dc=False, scalings = 'auto', block=True, color=dict(seeg='b'),
         events=events_array_orig, event_id=events_id_orig, event_color=event_color_orig)

Using qt as 2D backend.
Channels marked as bad:
none


In [14]:
epochs = mne.Epochs(mne_data, events_array_orig, tmin=-0.5, tmax=0.5, event_id=events_id_orig, preload=True)
epochs.plot_image(combine="mean")

Not setting metadata
939 matching events found
Setting baseline interval to [-0.5, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 939 events and 1001 original time points ...
0 bad epochs dropped
Not setting metadata
939 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_146655/3756777855.py:2: RuntimeWarning: Only one channel in group "seeg"; cannot combine by method "mean".
  epochs.plot_image(combine="mean")


combining channels using "mean"


/tmp/ipykernel_146655/3756777855.py:2: RuntimeWarning: Only 1 channel in "picks"; cannot combine by method "mean".
  epochs.plot_image(combine="mean")


[<Figure size 640x480 with 3 Axes>]

In [15]:
min_freq = 65
max_freq = 135
freqs = np.arange(min_freq, max_freq+2, 2)  # 
n_cycles = freqs/500
t_win = n_cycles / freqs

power = mne.time_frequency.tfr_multitaper(
    epochs,
    freqs = freqs,
    n_cycles = freqs,
    use_fft = True,
    return_itc = False,
    average = False,
    decim = 2,
)

Not setting metadata


In [16]:
#power['putative_ripple'].average().plot()
power.apply_baseline(baseline=(-0.5, -0.2), mode="zscore").average().plot()

Applying baseline correction (mode: zscore)
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
No baseline correction applied


[<Figure size 640x480 with 2 Axes>]

In [17]:
freqs = np.logspace(*np.log10([60, 160]), num=30)
n_cycles = freqs / 2.0  # different number of cycle per frequency
power, itc = mne.time_frequency.tfr_morlet(
    epochs,
    freqs=freqs,
    n_cycles=n_cycles,
    use_fft=True,
    return_itc=True,
    decim=3,
    n_jobs=None,
)

In [18]:
power.plot(baseline=(-0.7, -0.2), mode="logratio")

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Applying baseline correction (mode: logratio)


[<Figure size 640x480 with 2 Axes>]

In [23]:
importlib.reload(SpikeWaveDetectorClass)

<module 'SpikeWaveDetectorClass' from '/home/lfa-01/Documentos/RepositoriosGit/ripple_IIS_detection/source/SpikeWaveDetectorClass.py'>

In [16]:
ieee_signal.shape

(1, 28799616)

In [19]:
zs_amp_all = stats.zscore(ieee_signal, axis=1) 
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(times, ieee_signal.T)
plt.xlabel('Time')
plt.ylabel('Ampl')
plt.title('Signal')
plt.show()

# Plot zscore along time
plt.figure(figsize=(10, 5))
plt.plot(times, zs_amp_all.T)
plt.xlabel('Time')
plt.ylabel('Zscore')
plt.title('Zscore along Time')
plt.show()

# Plot histogram
plt.figure(figsize=(10, 5))
plt.hist(zs_amp_all.flatten(), bins=50)
plt.xlabel('Zscore')
plt.ylabel('Frequency')
plt.title('Histogram of Zscore')
plt.show()

print(zs_amp_all)
print(np.count_nonzero(np.isnan(ieee_signal)))

[[ 1.43052334e-02 -5.01535413e-04 -3.57857506e-02 ...  1.57342652e+00
   1.55924984e+00  1.55168891e+00]]
0


In [29]:
# Instantiate SpikeWaveDetectorClass
IIS_det = SpikeWaveDetectorClass.SpikeWaveDetectorClass(int(fsample))

In [30]:
#peak_time, passed_condition = IIS_det.detect_times(ieee_signal, True)
peak_times, peak_stats = IIS_det.detect_times(ieee_signal, True)

In [31]:
keys = peak_stats.keys()
print(keys)

print(len(peak_stats['zscores_per_peaks_max']))
print(len(peak_stats['zscores_per_peaks_env']))
print(len(peak_stats['zscores_per_peaks_amp']))
print(len(peak_stats['zscores_per_peaks_grad']))
print(len(peak_stats['inds_per_peak']))
print(len(peak_stats['passed_conditions']))

print(type(peak_stats['zscores_per_peaks_max']))
print(type(peak_stats['zscores_per_peaks_env']))
print(type(peak_stats['zscores_per_peaks_amp']))
print(type(peak_stats['zscores_per_peaks_grad']))
print(type(peak_stats['inds_per_peak']))
print(type(peak_stats['passed_conditions']))

dict_keys(['zscores_per_peaks_max', 'zscores_per_peaks_env', 'zscores_per_peaks_amp', 'zscores_per_peaks_grad', 'inds_per_peak', 'passed_conditions'])
1283
1283
1283
1283
1283
1283
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [23]:
plot_z_scores = True
IIS_det.plot_spike_waves(ieee_signal, peak_times, 1, peak_stats, plot_z_scores)  

In [32]:
# Assume you have a dictionary called 'my_dict'
my_dict = {'key1': 'value1', 'key2': 'value2', 'key3': {'nested_key1': 'nested_value1', 'nested_key2': 'nested_value2'}}

# Get the keys of the dictionary
keys = my_dict.keys()

# Print the keys
print(keys)

# Access the values associated with each key
for key in keys:
    value = my_dict[key]
    print(f"Key: {key}, Value: {value}")
type(peak_stats)

dict_keys(['key1', 'key2', 'key3'])
Key: key1, Value: value1
Key: key2, Value: value2
Key: key3, Value: {'nested_key1': 'nested_value1', 'nested_key2': 'nested_value2'}


dict

## Deteccion de Ripples

In [33]:
importlib.reload(RippleDetectorClass)

<module 'RippleDetectorClass' from '/home/lfa-01/Documentos/RepositoriosGit/ripple_IIS_detection/source/RippleDetectorClass.py'>

In [34]:
# Instantiate RippleDetectorClass
Ripp_det = RippleDetectorClass.RippleDetectorClass(int(fsample))

In [35]:
ripple_times, ripple_start_end = Ripp_det.detect_ripple(ieee_signal, None, peak_times, None)

In [36]:
len(ripple_times)

2397

In [15]:
Ripp_det.plot_ripples(ieee_signal, ripple_times, None)

In [37]:
events_array = np.column_stack((np.concatenate((ripple_times, peak_times)), np.zeros(len(ripple_times)+len(peak_times)), np.concatenate((2*np.ones(len(ripple_times)),3*np.ones(len(peak_times))))))
#events_array = np.column_stack((ripple_times, np.zeros(len(ripple_times)), np.ones(len(ripple_times))))

events_array = events_array.astype(int)
print(events_array)

[[     337        0        2]
 [   28198        0        2]
 [   32776        0        2]
 ...
 [28722897        0        3]
 [28725653        0        3]
 [28725845        0        3]]


In [38]:
#Creacion de canal para incluir la informacion de eventos
#Originalmente es una senal digital que se transforma a analogica para crear el dataset de mne
stim_chann = np.zeros((1, len(ieee_signal[0])))
info_stim = mne.create_info(['STIM'], mne_data.info['sfreq'], ['stim'])
stim_raw = mne.io.RawArray(stim_chann, info_stim)
mne_data.add_channels([stim_raw], force_update_info=True)

Creating RawArray with float64 data, n_channels=1, n_times=28799616
    Range : 0 ... 28799615 =      0.000 ... 28799.615 secs
Ready.


<RawArray | 2 x 28799616 (28799.6 s), ~439.5 MB, data loaded>

In [39]:
# Se agrega la informacion de los eventos como canal STIM
mne_data.add_events(events_array, stim_channel='STIM', replace=True)

Se contatenan los eventos originales de Ngo con los eventos nuevo obtenidos por mi

In [40]:
events_array_full = np.vstack((events_array_orig, events_array))
print(events_array_full) 

[[    3832        0        1]
 [    7052        0        1]
 [   28194        0        1]
 ...
 [28722897        0        3]
 [28725653        0        3]
 [28725845        0        3]]


In [41]:
events_id = {'putative_ripple_ngo': 1, 'putative_ripple': 2, 'putative_IIS': 3}
event_color = {'putative_ripple_ngo': 'm', 'putative_ripple': 'r', 'putative_IIS': 'k'}

#events_id = {'putative_ripple': 1, 'putative_IIS': 2}
#event_color = {'putative_ripple': 'r', 'putative_IIS': 'k'}

In [44]:
#seeg_channels = mne.pick_types(mne_data.info, seeg=True)
mne_data.plot(events=events_array_full, event_id=events_id, event_color=event_color, duration=10, order=seeg_channels, n_channels=len(seeg_channels),
         remove_dc=False, scalings = 'auto', block=True, color=dict(seeg='b'))

Channels marked as bad:
none


In [43]:
zs_amp_all = stats.zscore(ieee_signal, axis=1) 
print(zs_amp_all.shape)

(1, 28799616)


In [33]:
points_in_block = 30000
n_blocks = 720
data = ieee_signal
print(data[0].shape)

for i_block in range(1, n_blocks + 1):
    curr_block = data[0][(i_block - 1) * points_in_block:i_block * points_in_block]
    n_curr_block = len(curr_block)
    #print(n_curr_block)
    data_gradient = np.concatenate([[0], np.diff(curr_block)])
    print(data_gradient.shape)


(21600384,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,)
(30000,

In [25]:
len(ieee_signal[0])

21600384

In [5]:
IIS_det.useConjAmpEnv

False

In [69]:
ieee_signal = mat_data['trial'][0][0][0]
print(len(ieee_signal))

ch_types=['seeg']*cant_chan
print(ch_types)

28483712
['seeg']


In [12]:
import h5py
import numpy as np
# Load the .mat file

# FILEPATH: Untitled-1.ipynb
filepath = '/home/lfa-01/Descargas/pat01_HIPP.mat'
f = h5py.File(filepath)

arrays = {}
for k, v in f.items():
    arrays[k] = np.array(v)

type(arrays)
print(arrays.keys())


dict_keys(['#refs#', 'fsample', 'label', 'sampleinfo', 'time', 'trial'])


In [23]:
time_data = arrays['fsample']
print(time_data)


[[1000.]]


In [25]:
type(arrays['time'])

numpy.ndarray

In [ ]:
# Convert the loaded data to a MNE-Python object
mne_data = mne.io.RawArray(mat_data['data'], info=mne.create_info(ch_names=mat_data['channel_names'], sfreq=mat_data['sampling_rate']))

# Now you can work with the MNE-Python object